In [ ]:
import os


In [3]:
%pwd


'c:\\Github\\Text-Summarization\\research'

In [4]:
os.chdir("c:\\Github\\Text-Summarization")
%pwd


'c:\\Github\\Text-Summarization'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int


In [ ]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [ ]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.root_dir])

        return data_transformation_config

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_Trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(

            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=config.num_train_epochs,
            warmup_steps=config.warmup_steps,
            per_device_train_batch_size=config.per_device_train_batch_size,
            weight_decay=config.weight_decay,
            logging_steps=config.logging_steps,
            evaluation_strategy=config.evaluation_strategy,
            eval_steps=config.eval_steps,
            save_steps=config.save_steps,
            gradient_accumulation_steps=config.gradient_accumulation_steps
        )
        return model_trainer_config


In [ ]:
from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch


c:\Users\X1 Yoga\.conda\envs\mlproj\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-12-09 14:09:51,590: INFO: config PyTorch version 2.5.1 available.]


In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        """
        Trains the sequence-to-sequence model using the specified configuration.
        This method sets up the device (CPU or GPU), loads the tokenizer and model,
        prepares the data collator, loads the dataset, and configures the training
        arguments before starting the training process.
        Attributes:
            device (str): The device to use for training ('cuda' if GPU is available, otherwise 'cpu').
            tokenizer (AutoTokenizer): The tokenizer loaded from the specified model checkpoint.
            model_pegasus (AutoModelForSeq2SeqLM): The sequence-to-sequence model loaded from the specified model checkpoint.
            seq2seq_data_collator (DataCollatorForSeq2Seq): The data collator for sequence-to-sequence tasks.
            dataset_samsum_pt (Dataset): The dataset loaded from the specified data path.
            trainer_args (TrainingArguments): The training arguments configured with the specified parameters.
        Returns:
            None
        """
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(
            self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(
            tokenizer, model=model_pegasus)
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir="pegasus-samsum",
            num_train_epochs=1,
            warmup_steps=500,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            weight_decay=0.01,
            logging_steps=10,
            evaluation_strategy='steps',
            eval_steps=500,
            save_steps=1e6,
            gradient_accumulation_steps=16)

        # trainer_args = trainerArguments(
        #     output_dir=self.config.root_dir,
        #     num_train_epochs=self.config.num_train_epochs,
        #     warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size,
        #     per_device_eval_batch_size=self.config.per_device_eval_batch_size,
        #     weight_decay=self.config.weight_decay,
        #     logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy,
        #     eval_steps=self.config.eval_steps,
        #     save_steps=self.config.save_steps,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps)


In [ ]:

trainer = Trainer(model=model_pegasus,
                  arg=trainer_args,
                  tokenizer=tokenizer, data_collector=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt['validation'])


NameError: name 'model_pegasus' is not defined

In [ ]:
trainer.train()


NameError: name 'trainer' is not defined